# ReGroup Demo

Welcome to ReGroup demo Jupyter presentation. This demo consists of two parts which can be run separately. 

Part I concerns symmetry determination, correction and symmetrization of some points in n-dimensional space. Presumably those points can be seen as  vertices of a distorted polytope.

Part II allows to find and correct molecular symmetry and symmetrize a molecule; Part II uses nglview to visualize molecule and symmetry.

# Part I. Finding & correcting the symmetry of a distorted polytope

## 1. Select polytope type:
Run the code below to see selection widget

In [ ]:
import ipywidgets as wdg
import numpy as np
from IPython.display import display, clear_output, Markdown
import polytope as ptp
from math import sqrt
from symmfinder import symmetry_finder, inclusive_closure
from grpcorr import multab_group_correction

In [ ]:

pchoice = wdg.RadioButtons( options=["hypercube","hypercube dual polytope","simplex",
                                    "dodecahedron (3D)","icositetrachoron (4D)", "hexacosichoron (4D)"], 
                           description = "Polytope")
dchoice = wdg.IntSlider(min=1, max=7, disabled=False, value = 1, description = 'Dimension')

def dchoice_update(*args):
    off = False
    if pchoice.index == 3:
        dchoice.value=3 
        dchoice.disabled = True
    elif pchoice.index in [4,5]:
        dchoice.value=4
        dchoice.disabled = True
    else:
        dchoice.disabled = False

def print_ptope(x,y):
    print("You selected: ",x,", D = ", y)
        
pchoice.observe(dchoice_update,'index')
        
wdg.interactive(print_ptope,x=pchoice,y=dchoice)

## 2. Enter polytope radius:
Run the code below to see the input widget, make sure you did not run the code above after selection of polytope type, otherwise select it again.

By polytope radius we mean the distance between its center and vertices.

In [ ]:
rchoice = wdg.BoundedFloatText(description = "Radius", step = .05, min = 0e0)
display(rchoice)

## 3. Random distortion:
Now you can randomly displace all the points along all coordinate axes. Provide the displacement amplitude in the widget below (run the code to see the widget)

In [ ]:
distort = wdg.Checkbox(description = "Apply random distortion", value = True)
epschoice = wdg.BoundedFloatText(description = "Amplitude", step = .05, min = 0e0)

def epschoice_update(*args):
    epschoice.disabled = not distort.value
    if not distort.value:
        epschoice.value = 0e0
    
distort.observe(epschoice_update,"value")

def print_distort(d,eps):
    print("Random distortion: ",d,"; amplitude = ", eps)
    
wdg.interactive(print_distort, d=distort, eps = epschoice)

## 4. Generate the points:
In the widgets below you can do one of two possible things: 1) generate the points according the choice you made; 2) copy and paste your own points from other source

In [ ]:
bcreate = wdg.Button(description = "Create polytope")
bclear = wdg.Button(description = "Clear points")
points_input = wdg.Textarea(layout=wdg.Layout(width='100%'))

def create_polytope(*args):
    i = pchoice.index
    dim = dchoice.value
    a = rchoice.value
    eps = epschoice.value
    if i==0:
        points = ptp.hypercube(2*a/sqrt(dim),dim)
    elif i==1:
        points = ptp.hypercube_dual(a,dim)
    elif i==2:
        points = ptp.simplex(a,dim)
    elif i==3:
        points = ptp.dodecahedron(a)
    elif i==4:
        points = ptp.icositetrachoron(a)
    elif i==5:
        points = ptp.hexacosichoron(a)
    else:
        pass
    if distort.value:
        ptp.random_distortion(points,eps)
    points_input.value = '\n'.join([('{:10.6f}'*len(p)).format(*p) for p in points])
    
def clear_points(*args):
    points_input.value = ""
    
bcreate.on_click(create_polytope)
bclear.on_click(clear_points)
display(wdg.HBox([bcreate,bclear]))
display(Markdown("## 5. Use generated points or (alternatively) copy&paste them here \
\nFormat: n columns, space separated, where n is the space dimension"))
display(points_input)

## 6. Select epsilon (tolerance) value for approximate symmetry operations

In [ ]:
eps1choice = wdg.BoundedFloatText(description = "Tolerance", step = .05, min = 0e0)
display(eps1choice)

## 7. Find approximate symmetry operations
When points are created (step 5) and epsilon value is chosen, run the following step manually. It will find as many approximate symmetry operations as it can. The symmetry operations will be collected in the list G and corresponding points permutations - in the list P.

In [ ]:
points = [np.array([float(s) for s in l.split()]) for l in points_input.value.split('\n')]
dim = dchoice.value
eps = eps1choice.value
print(dim,eps)
P,G = symmetry_finder(points,dim,eps)

## 8. Perform closure
If the points were displaced from symmetric configuration, it is highly likely that symmetry finder was not able to find all symmetry operations. In this case the set of operations found now is not a group since it is not closed with respect to multiplication. The following code will recover missing operations and build group multiplication table.

**Warning: if the group size exceeds 1000 this might be time and memory consuming**

In [ ]:
Multab = inclusive_closure(P,G)

## 9. Multiplication table based group correction
Now it is time to apply the group correction algorithm and turn approximate symmetry operation into exact ones

In [ ]:
multab_group_correction(G,Multab,eps=1e-10)

## 10. Results
You can see the group matrices one by one (which may be not very meaningful for large group) or save the group as a list of numpy arrays to a file

In [ ]:
def mtr_str(g):        
    return '\n'.join([("{:8.5f} "*g.shape[1]).format(*g[i,:]) for i in range(g.shape[0])])
        
mchoice = wdg.IntSlider(min=0, max=len(G)-1, disabled=False, value = 0)                         
ichoice = wdg.BoundedIntText(min=0, max=len(G)-1, disabled=False, value = 0, 
                             description = 'SymmOp#')
mshow = wdg.Textarea(layout=wdg.Layout(width='100%',height='100%'))
def mchoice_update(*args):
    i = ichoice.value
    mchoice.value = i
    mshow.value=mtr_str(G[i])
    
def ichoice_update(*args):
    i = mchoice.value
    ichoice.value = i
    mshow.value=mtr_str(G[i])
    
mchoice.observe(ichoice_update,'value')
ichoice.observe(mchoice_update,'value')
display(wdg.VBox([ichoice,mchoice]),mshow)


## 11. Save the group
Provide filename to save of leave it empty to output it below

In [ ]:
fname = wdg.Text(description = 'file to save:', value = '')
bsave = wdg.Button(description = 'Save')

def save_group(*args):
    if fname.value == '':
        print(G)
    else:
        f = open(fname.value)
        print(G,file = f)
        f.close()

bsave.on_click(save_group)
display(fname,bsave)

## 12. Symmetrize the points
The points can be corrected back to symmetric configuration

In [ ]:
points1 = ptp.symmetrize_points(points,P,G)
print('\n'.join([('{:10.6f}'*len(p)).format(*p) for p in points1]))



# Part II. Application to molecular symmetry 

In [ ]:
import nglview as ngl
from ase import Atom, Atoms
import ipywidgets as wdg
import numpy as np
from IPython.display import display, clear_output, Markdown
import polytope as ptp
from symmfinder import symmetry_finder, inclusive_closure
from grpcorr import multab_group_correction

## 1. Read molecule from xyz file
The xyz file format is simple:

*A1 x1 y1 z1*

*A2 x2 y2 z2*

............

*An xn yn zn*

Here *Ai* is atomic symbol and *xi, yi, zi* are atomic coordinates. First two lines might also contain the number of atoms and some comment, however, it is not necessary. The limited number of ready-to-go examples includes "examples/c20.xyz", "examples/ch4.xyz", "examples/c2h6.xyz" and "examples/sf6.xyz".

In [ ]:
fname = wdg.Text(description = 'xyz file:', value = 'examples/c20.xyz')
bopen = wdg.Button(description = 'Open')
mol_input = wdg.Textarea(layout=wdg.Layout(width='70%'))

def open_xyz_file(*args):
    f = open(fname.value)
    mol_input.value = ''.join([l for l in f])
    f.close()

bopen.on_click(open_xyz_file)
display(wdg.HBox([fname,bopen]),Markdown('##  2. Alternatively, copy & paste it here'),mol_input)

## 3. Apply random distortion

In [ ]:
distort = wdg.Checkbox(description = "Apply random distortion", value = True)
epschoice = wdg.BoundedFloatText(description = "Amplitude", step = .05, min = 0e0)

def epschoice_update(*args):
    epschoice.disabled = not distort.value
    if not distort.value:
        epschoice.value = 0e0
    
distort.observe(epschoice_update,"value")

def print_distort(d,eps):
    print("Random distortion: ",d,"; amplitude = ", eps)
    
wdg.interactive(print_distort, d=distort, eps = epschoice)

## 4. Read & visualize initial molecule
Manually run the following code

In [ ]:
lines=[line.split() for line in mol_input.value.split('\n')]
mol = {'atoms':[],'coord':[]}
for l in lines:
    if len(l)==4:
        mol['atoms'].append(l[0])
        mol['coord'].append(np.array([float(l[i]) for i in [1,2,3]]))
        
if distort.value:
    ptp.random_distortion(mol['coord'],epschoice.value)

ase_mol = Atoms([Atom(mol['atoms'][i], tuple(mol['coord'][i])) for i in range(len(mol['atoms']))])
mol_view = ngl.show_ase(ase_mol)
display(mol_view)

## 5. Select epsilon (tolerance) value for approximate symmetry operations

In [ ]:
eps1choice = wdg.BoundedFloatText(description = "Tolerance", step = .05, min = 0e0)
display(eps1choice)

## 6. Find approximate symmetry group

In [ ]:
eps = eps1choice.value
P,G = symmetry_finder(mol['coord'],3,eps)
Multab = inclusive_closure(P,G)

def generate_arrows(G):
    colors = {(2,False):[0,0,0], (3,False):[0,1,0], (4,False):[0,0,1], 
                (5,False):[1,0,0], (6,False):[1,1,0], (2,True):[1,1,1],
                (4,True):[1,0,1],(6,True):[0, 1, 1]}
    rmax = max([np.linalg.norm(r) for r in mol['coord'] ])
    res = []
    gdat = []
    for g in G:
        gdata = ptp.analyze_matrix3(g,eps/rmax)
        gdat.append(gdata)
        b = 1.3*rmax*gdata['axis']
        n = int(gdata['order']+.5)
        i = gdata['inversion']
        if n>1 and abs(n-gdata['order'])<eps/rmax and (n,i) in colors:
            res.append([list(-b),list(b),colors[n,i],.1])
    return res, gdat
        
ase_mol = Atoms([Atom(mol['atoms'][i], tuple(mol['coord'][i])) for i in range(len(mol['atoms']))])
mol_view = ngl.show_ase(ase_mol)
arr = generate_arrows(G)
for x in arr[0]:
    mol_view.shape.add_arrow(*x)
display(Markdown('*Axes colors denote their order: black - 2, \
                 green - 3, blue - 4, red - 5, yellow - 6, reflection - white, \
                 rotoinversion 4 - magenta, rotoinversion 6 - cyan*'))
display(mol_view)
for i in range(len(arr[1])):
    x = arr[1][i]
    if x['inversion']:
        print("SymmOp#{:5d} order={:6.4f}  axis=({:8.5f},{:8.5f},{:8.5f}) inversion".
              format(i,x['order'],*x['axis']) )
    else:
        print("SymmOp#{:5d} order={:6.4f}  axis=({:8.5f},{:8.5f},{:8.5f})".
              format(i,x['order'],*x['axis']) )

## 7. Apply multiplication table based correction and symmetrize molecule

In [ ]:
def symmetrize_mol(mol,G,P):
    mol1 = [np.array([0e0,0e0,0e0]) for m in mol['coord']]
    for i in range(len(G)):
        for j in range(len(mol['coord'])):
            k = P[i][j]
            mol1[k] = mol1[k] + G[i].dot(mol['coord'][j])
    for i in range(len(mol1)):
        mol1[i] /= len(G)
    mol['coord'] = mol1
    return mol

def print_xyz(mol):
    print(len(mol['atoms']))
    print()
    for i in range(len(mol['atoms'])):
        print('{:2s} {:10.5f} {:10.5f} {:10.5f}'.format(mol['atoms'][i],*mol['coord'][i]))

multab_group_correction(G,Multab,eps=1e-12)
mol = symmetrize_mol(mol,G,P)
display(Markdown('### Symmetrized molecule in xyz format:'))
print_xyz(mol)    

## 8. Visualize symmetrized molecule

In [ ]:
ase_mol_sym = Atoms([Atom(mol['atoms'][i], tuple(mol['coord'][i])) for i in range(len(mol['atoms']))])
mol_view_sym = ngl.show_ase(ase_mol_sym)
for x in generate_arrows(G)[0]:
    mol_view_sym.shape.add_arrow(*x)
display(Markdown('Axes color denotes its order: black - 2, \
                 green - 3, blue - 4, red - 5, yellow - 6, reflection - white, \
                 rotoinversion 4 - magenta, rotoinversion 6 - cyan'))
display(mol_view_sym)

# Postscriptum

The <code>ReGroup</code> library whose functionality is demonstrated in this Jupyter presentaion is able to find and correct the symmetry group for any object provided that its symmetry can be expressed with unitary matrices. If this object is distorted from its symmetric configuration, it can be restored when symmetry group is known. 

Many other programs and libraries operating with molecular calculations data can automatically determine the molecular symmetry comparing it with a predefined list of known symmetry groups. Unlike them, this code does not rely on any predefined groups, instead, it builds symmetry group from scratch. For point groups in 3D space this might seem unneccessary, but for higher dimension or for non-point symmetries this approach can be highly useful.
    
The efficiency of present implementation can be greatly improved by re-coding a part of it in C++. But in present edition it is not done deliberately to make its use easier. For large groups great acceleration can be achieved by using group generators instead of entire group, but this can be done if the future if the need in such methods appears.    
    
There more examples of <code>ReGroup</code> applications. The directory 'examples' contains Python scripts:
<ul>
<li><code>create-polytope.py</code>  -  create and save points </li>
<li><code>example-multidim.py</code> -  read points from file, apply symmetry finder, inclusive closure and multiplication table based group correction</li>
<li><code>example-mol.py</code>      -  read a molecule from file, find approximate symmetry and make it exact, including possible rotation of the group</li>
</ul>
